In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)


TensorFlow: 2.20.0


In [2]:
BASE_DIR = r"E:\ML\Projects\pneumonia_detector\data"

train_dir = os.path.join(BASE_DIR, "train")
val_dir   = os.path.join(BASE_DIR, "val")
test_dir  = os.path.join(BASE_DIR, "test")

print("Train:", train_dir)
print("Val:", val_dir)
print("Test:", test_dir)

print("Train folders:", os.listdir(train_dir))
print("Val folders:", os.listdir(val_dir))
print("Test folders:", os.listdir(test_dir))


Train: E:\ML\Projects\pneumonia_detector\data\train
Val: E:\ML\Projects\pneumonia_detector\data\val
Test: E:\ML\Projects\pneumonia_detector\data\test
Train folders: ['NORMAL', 'PNEUMONIA']
Val folders: ['NORMAL', 'PNEUMONIA']
Test folders: ['NORMAL', 'PNEUMONIA']


In [7]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)


val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

print("Class indices:", train_data.class_indices)

Found 4434 images belonging to 2 classes.
Found 798 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Class indices: {'NORMAL': 0, 'PNEUMONIA': 1}


In [8]:
from tensorflow.keras import layers
from tensorflow.keras import models


base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

# Freeze most layers
for layer in base_model.layers[:-80]:
    layer.trainable = False

# Unfreeze last layers
for layer in base_model.layers[-80:]:
    layer.trainable = True


inputs = keras.Input(shape=(224,224,3))

x = base_model(inputs, training=True)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(
    128,
    activation="relu",
    kernel_regularizer=tf.keras.regularizers.l2(0.001)
)(x)

x = layers.Dropout(0.4)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 2,231,809 (8.51 MB)

 Non-trainable params: 190,272 (743.25 KB)

In [9]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)


In [10]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stop]
)


Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.8083 - loss: 0.6474 - val_accuracy: 0.7669 - val_loss: 0.6148
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.9145 - loss: 0.4567 - val_accuracy: 0.7920 - val_loss: 0.6022
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 259s 2s/step - accuracy: 0.9369 - loss: 0.3999 - val_accuracy: 0.8070 - val_loss: 0.5944
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.9497 - loss: 0.3705 - val_accuracy: 0.8546 - val_loss: 0.5051
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.9590 - loss: 0.3414 - val_accuracy: 0.8772 - val_loss: 0.4602
Epoch 6/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9569 - loss: 0.3385 - val_accuracy: 0.9373 - val_loss: 0.3729
Epoch 7/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.9644 - loss: 0.3206 - val_accuracy: 0.9511 - val_loss: 0.3362
Epoch 8/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.9608 - loss: 0.3276 - val_accu

In [11]:
model.evaluate(test_data)

20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 910ms/step - accuracy: 0.8365 - loss: 0.7052


[0.7052453756332397, 0.8365384340286255]

In [15]:
from PIL import Image
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


def predict_image(path):
    img = Image.open(path).convert("RGB")
    img = img.resize((224,224))

    arr = np.array(img, dtype=np.float32)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)

    pred = model.predict(arr, verbose=0)[0][0]
    return float(pred)

In [16]:
print("NORMAL:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\NORMAL\IM-0091-0001.jpeg"))
print("PNEUMONIA:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\PNEUMONIA\person67_virus_126.jpeg"))
print("NORMAL:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\NORMAL\IM-0061-0001.jpeg"))
print("PNEUMONIA:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\PNEUMONIA\person47_virus_99.jpeg"))
print("NORMAL:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\NORMAL\NORMAL2-IM-0066-0001.jpeg"))
print("PNEUMONIA:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\PNEUMONIA\person83_bacteria_407.jpeg"))
print("NORMAL:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\NORMAL\IM-0067-0001.jpeg"))
print("PNEUMONIA:", predict_image(r"E:\ML\Projects\pneumonia_detector\data\test\PNEUMONIA\person78_bacteria_378.jpeg"))

NORMAL: 0.8869919776916504
PNEUMONIA: 0.998544454574585
NORMAL: 0.068330779671669
PNEUMONIA: 0.9999887943267822
NORMAL: 0.9688714146614075
PNEUMONIA: 0.9949080944061279
NORMAL: 0.33069393038749695
PNEUMONIA: 0.9999448657035828


In [17]:
model.save(
    "../deploy/model/pneumonia_model.keras",
    include_optimizer=False
)